In [31]:
# Importing core libraries required for the case study
import numpy as np
import pandas as pd
import matplotlib.pyplot as plot
import seaborn as sea
import datetime as dt
import warnings

loan = pd.read_csv('./dataset/loan.csv', low_memory=False)

loan.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
0,1077501,1296599,5000,5000,4975.0,36 months,10.65%,162.87,B,B2,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
1,1077430,1314167,2500,2500,2500.0,60 months,15.27%,59.83,C,C4,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
2,1077175,1313524,2400,2400,2400.0,36 months,15.96%,84.33,C,C5,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
3,1076863,1277178,10000,10000,10000.0,36 months,13.49%,339.31,C,C1,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
4,1075358,1311748,3000,3000,3000.0,60 months,12.69%,67.79,B,B5,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN


# Data Cleaning and Manipulation
Aiming to clean and manipulate the data to finally reach a data form where we can work with the data. Will wilcude the following steps while cleaning and manipulating data.
1. Find the number of columns which do not have any data.
2. Drop unwanted and duplicate rows.
3.


## Getting the summary of all columns which contain null values
Print summary of Nulls, Blanks in the dataset the ine form a percentage value.

In [32]:
(loan.isnull().sum()/len(loan.index) * 100)

id                              0.000000
member_id                       0.000000
loan_amnt                       0.000000
funded_amnt                     0.000000
funded_amnt_inv                 0.000000
                                 ...    
tax_liens                       0.098195
tot_hi_cred_lim               100.000000
total_bal_ex_mort             100.000000
total_bc_limit                100.000000
total_il_high_credit_limit    100.000000
Length: 111, dtype: float64

## Removing All Rows With loan_status = "Current"
Loan Status - Key Leading Attribute (loan_status). The column has three distinct values
- Fully-Paid - The customer has successfuly paid the loan
- Charged-Off - The customer is "Charged-Off" ir has "Defaulted"
- Current - These customers, the loan is currently in progress and cannot contribute to conclusive evidence if the customer will default of pay in future
**For the given case study, "Current" status rows will be ignored**

In [38]:
prev_rows = len(loan)
# The rows where loan_stats=Current are the data where the loan repayment is currently in progress
# The loans which are currently in progress will not contribute to decisions
# of default or pass as it's difficult to predict the outcome
#
# Dropping the rows early as, dropping all Current rows introduces NA columns which can be easily dropped
loan = loan[loan['loan_status'] != "Current"]

# Print current data statistics after dropping rows with loan_status "CURRENT"
curr_rows = len(loan)
print(curr_rows)
print(prev_rows)

print("Number of rows dropped where loan_status = 'Current':", (prev_rows - curr_rows))
print("Percentage of rows dropped = ", round((prev_rows - curr_rows)/prev_rows*100,2),"%")

38577
38577
Number of rows dropped where loan_status = 'Current': 0
Percentage of rows dropped =  0.0 %


## Find any duplicate rows in the dataset

In [35]:
duplicate_rows = len(loan[loan.duplicated()])
if duplicate_rows <= 0:
    print("Duplicate Rows: ", duplicate_rows)
    print("No action needed")
else:
    print(len(duplicate_rows))

Duplicate Rows:  0
No action needed


### Dropping Columns That Have ID Values
Dropping columns which is unique id in nature. They dont contribute to loan analysis

In [39]:
# Checking if member_id is unique
if len(loan['member_id'].unique()) == loan.shape[0]:
    print("member_id is unique, can be dropped")
    loan = loan.drop(['member_id'],  axis=1)
else:
    print("member_id is not unique, dont drop")

# Checking if id is unique
if len(loan['id'].unique()) == loan.shape[0]:
    print("id is unique, can be dropped")
    # not dropping id as it will be used for pivot calculation later
    # loan = loan.drop(['id'],  axis=1)
else:
    print("id is not unique, dont drop")

member_id is unique, can be dropped
id is unique, can be dropped


### Dropping text/description columns which will not contribute to overall analysis
These are names of establishment etc which will not contribute to loan pass or failure. The URL column is a static link with id as the attribute. It's a redundant column

In [ ]:
loan = loan.drop(['url', 'emp_title', 'desc', 'title'],  axis=1)

# Dropping column sub_grade as the current analysis will limit to Grade only

In [40]:
loan = loan.drop(['sub_grade'],  axis=1)

### Dropping all columns which refer to behavioural data of customer post loan approval
Behaviour data of the customers are captured post the loan approval. The data is not available at the time of loan approval and thus cannot be used for calculations